# WARNING

Genetic algorithms are not for the faint of heart. It is a difficult algorithm with many parameters that do many different things, and all affect the final solution in a meaningful way. The code for genetic algorithms is also very complex. So **if you are not comfortable with progrmming or with the complexity of genetic algorithms you should ignore this notebook** and stick to something easier like randomized search or grid search.

Again, just as a final warning, this notebook is very advanced. If you don't understand, then just ignore this notebook.


In [ ]:
import pandas as pd
import numpy as np

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split

In [ ]:
perth = pd.read_csv('perth_clean.csv')
perth = pd.get_dummies(perth, columns=['suburb'])

train_indices, test_indices = train_test_split(perth.index, test_size=0.2, random_state=0)

perth_train = perth.loc[train_indices].copy()
perth_test = perth.loc[test_indices].copy()

In [ ]:
indices = list(perth_train.index.copy())
# np.random.seed(5)
np.random.shuffle(indices)

sampled_indices = indices[:int(len(indices) * 0.2)]
sampled_perth = perth_train.loc[sampled_indices, :].copy()

print("Sampled set length", len(sampled_indices))

Sampled set length 5384


In [ ]:
n = len(sampled_perth)
sampled_train_indices, sampled_valid_indices = train_test_split(np.arange(n), test_size=0.2, random_state=0)


sampled_x_train = sampled_perth.iloc[sampled_train_indices].drop('log10_price', axis=1)
sampled_y_train = sampled_perth.iloc[sampled_train_indices]['log10_price'].copy()

sampled_x_valid = sampled_perth.iloc[sampled_valid_indices].drop('log10_price', axis=1)
sampled_y_valid = sampled_perth.iloc[sampled_valid_indices]['log10_price'].copy()

In [ ]:
train_data = (sampled_x_train, sampled_y_train)
valid_data = (sampled_x_valid, sampled_y_valid)

In [ ]:
print("Training length of sampled data", len(sampled_x_train))
print("Validiation length of sampled data", len(sampled_x_valid))

Training length of sampled data 4307
Validiation length of sampled data 1077


# Genetic Algorithms in Practice

### Step 0.1 - Defining the DNA
Before we can do anything, genetic algorithms requires a population, a population requires people, and people require a DNA. So we must start with a DNA. The DNA IS a potential solution to the problem you have. In our case, since we want to find the optimal `min_samples_leaf` and `max_features` a DNA is simply
```
DNA1 = {'min_samples_leaf': 10, 'max_features': 2}  # or
DNA2 = {'min_samples_leaf': 3, 'max_features': 15}
```

### Step 0.2 - Defining the Mating Step
The next most important step of GA is the mating procedure, that is how we go from one generation to the next. To do this, we must take two parents and combine them into 1
```
parent1 = {'min_samples_leaf': 10, 'max_features': 2}
parent2 = {'min_samples_leaf': 3, 'max_features': 15}
```

Now depending on the problem you have, you may need to be more creative, but here we will just use the cross-over method. In the cross-over method the child simply is a random choice between the gene of parent 1 and parent 2. For example, let us consider two parents with 5 different parameters:

|  | Gene 1 | Gene 2 | Gene 3 | Gene 4 | Gene 5 |
| --- | --- | --- | --- | --- | --- |
| Parent 1 | 9 | 10 | 11 | 12 | 17 |
| Parent 2 | 7 | 3 | 16 | 8 | 1 |
| --- | --- | --- | --- | --- | --- |
| Child | Choose(9, 7) | Choose(10, 3) | Choose(11, 16) | Choose(12, 8) | Choose(17, 1) |

Where Choose(x, y) is simply a random choice between x and y, with 50\% chance for each. So in the cross over method, we simply randomly select one of the gene of the parent genes. But, you can be creative with this. Another one you can use is

|  | Gene 1 | Gene 2 | Gene 3 | Gene 4 | Gene 5 |
| --- | --- | --- | --- | --- | --- |
| Parent 1 | 9 | 10 | 11 | 12 | 17 |
| Parent 2 | 7 | 3 | 16 | 8 | 1 |
| --- | --- | --- | --- | --- | --- |
| Child | mean(9, 7) | mean(10, 3) | mean(11, 16) | mean(12, 8) | mean(17, 1) |

But this will have some issues, which I won't explain here. Or you can use something else you wish.

So, lets say that we did the cross-over method and this is what we get

|  | Gene 1 | Gene 2 | Gene 3 | Gene 4 | Gene 5 |
| --- | --- | --- | --- | --- | --- |
| Parent 1 | 9 | 10 | 11 | 12 | 17 |
| Parent 2 | 7 | 3 | 16 | 8 | 1 |
| --- | --- | --- | --- | --- | --- |
| Child | 9 | 10 | 16 | 12 | 1 |

The problem here is that the child, and therefore next generation, 

### Step 0.3 - Mutation
GA will be nothing if not for mutation, as mutation adds the important exploration to GAs. Here mutation is simple, since our DNA is `child = {'gene1': 9, 'gene2': 10, 'gene3': 16, 'gene4': 12, 'gene5': 1}`, mutation will simply just be to suggest a random number for each gene with a given proability $p$. 

Of course, a high $p$ would mean that you have a high exploration, but a low $p$ means you are more likely to exploit. The key to this parameter is that you want to start of with a high $p$, and as you go through generation after generation you want to start exploiting and decrease $p$.

### Step 1 - Creating A Generation
From here everything should be pretty simple. In this example we just optimize the random forest regressor, so the parameter grid is

```
parameter_grid = {'min_samples_leaf': np.arange(1, 100 + 1),
                  'max_features': np.arange(1, len(perth_train.columns) - 1, 5)}
```

Our initial population will simply be filled with random choices of `min_samples_leaf` and `max_features`.

### Step 2 - Ranking the Generation
Ranking is the easiest step. All we do is take each DNA and compute the RMSE, or cross-entropy if you are doing classification.

### Step 3 - Making the next Generation
We have already defined the mating procedure, i.e. taking 2 people and making a child. But we haven't yet defined how we choose the parents. Here I use a pretty simply definition, but more complex ones exist.

- **Truncation Selection**: Although basic, this is what I use here as it is good enough. In this method we simply take the top 50% of the population as the potential parents for the next population. From this top 50%, we then randomly choose 2 parents to make a child (+ mutation) until we have enough for our 2nd generation. Note that you can use less, like top 20-30%, but this will result in more exploitation and less exloration. So it might be good to start of with a high number early, but smaller and smaller as you go along.

- **Fitness proportionate selection, aka roulette wheel selection**: Here all people in the population has a chance to be parents for the next generation. But their chance is related to their fitness, so a person with a high fitness has a high chance of being a parent, but a person with low fitness has a lower chance. The benefits of doing this over truncation selection is that it will automatically perform exploration/exploitation so you don't have to do so. But it can be tricky to get working.

    The easiest definition for the probabiltiy is as follows: If $f_i$ is the score for the $i^\textrm{th}$ person in the population, and we want to **maximize** this score, we define the probability for this individual to be
    $$\large p_i = \frac{f_i}{\sum_i f_i}$$
    
    Importantly, this definition only works in you want to **maximize** the score. If you want to **minimize** the score you will need to use a different definition, which I won't do because it's a bit more complex.

### Step 3.1 - Elitism
There is one problem if we purely use this mating procedure to make the next generation: there is no gaurantee that the most fit DNA will continue into the next population, nor is there any gaurantee that the most fit DNA will be used as a parent to make the next generation.

To fix this we can introduce something knowns as elitism. Importantly, elitism is not always used, but typically is used. In elitism we simply take the top 1-3 fittess DNA of the current generation and simply put it into the next generation. This way we ensure that the next generation will have the best DNA.

In [1]:
def generate_next_population(current_population, train_data, valid_data, n_children=10, mutation=0.5):
    """ Given the `current_population` will create the next generation with population size `n_children`.
        The parents of next generation will be the top 50% of the `current_population` and the children
        has a mutation rate of `mutation`.
        
        Parameters
        ----------
        current_population : list of dict
            List DNAs, where the dictionary represents a DNA
        train_data : tuple
            Expected (x_train, y_train)
        valid_data : tuple
            Expected (x_valid, y_valid)
        n_children : int
            Number of children for the next generation
        mutation : float between 0-1
            Probabiltiy for mutation
    """
    print("Scoring Population")
    
    # Score the population and sort them
    population_scores = score_population(current_population, train_data, valid_data)
    sorted_population, sorted_scores = sort_population_by_scores(current_population, population_scores)

    # Take the top 50% of the population
    n = len(sorted_population) // 2
    top_population = sorted_population[:n]
    top_scores = sorted_scores[:n]
    
    # Make the next generation
    next_population = make_children_from_population(top_population, top_scores, n_children-1, mutation=mutation)
    next_population.append(sorted_population[0])  # Elitism
    
    print(f"Current Population Best Score {sorted_scores[0]}, with DNA {sorted_population[0]}")
    return next_population

In [ ]:
def score_population(population, train_data, valid_data):
    """ Scores the population on the given dataset 
    
        Parameters
        ----------
        population : list of dict
            List DNAs, where the dictionary represents a DNA
        train_data : tuple
            Expected (x_train, y_train)
        valid_data : tuple
            Expected (x_valid, y_valid)
    """
    population_scores = []
    n = len(population)
    for i, person in enumerate(population):
        base_model = RandomForestRegressor(criterion='squared_error', random_state=0, **person)
        score = score_model(base_model, train_data, valid_data)
        population_scores.append(score)
        
        print(f"{i + 1} of {n}: DNA {person} has score {score}")
    
    return population_scores

def score_model(model, train_data, valid_data):
    x_train, y_train = train_data
    x_valid, y_valid = valid_data
    
    model.fit(x_train, y_train)
    
    y_pred = model.predict(x_valid)
    return np.sqrt(np.mean((y_valid - y_pred) ** 2))

In [ ]:
def sort_population_by_scores(population, population_scores):
    """ Sort the population from smalles to highest 
    
        Parameters
        ----------
        population : list of dict
            List DNAs, where the dictionary represents a DNA
        population_scores : list of float
            List of the scores for the population.
            `population_scores[i]` is expected to be the score for `population[i]`.
    """
    sorted_population_with_scores = sorted(zip(population_scores, population))
    
    sorted_population = [person for score, person in sorted_population_with_scores]
    sorted_scores = [score for score, person in sorted_population_with_scores]

    return sorted_population, sorted_scores

In [ ]:
def make_children_from_population(population, population_scores, n_children, mutation=0.5):
    """ Given the population will make the next generation
    
        Parameters
        ----------
        population : list of dict
            List DNAs, where the dictionary represents a DNA
        population_scores : list of float
            Retired variable, no longer used since we are using truncation selection not 
            roulette wheel selection.
        n_children : int
            Number of children for the next generation
        mutation : float between 0-1
            Probabiltiy for mutation
    """
    
    children = []
    for i in range(n_children):
        parent1, parent2 = np.random.choice(population, 2, replace=False)
        
        child = generate_child(parent1, parent2)
        child = mutate_child(child, mutation=mutation)

        children.append(child)
        
    return children

def generate_child(person1, person2, p=0.5):
    """ Given two parents, will generate the child based on cross-over method 
    
        Paramaters
        ----------
        person1 : dict
        person2 : dict
        p : float
            The probability of using person1's genes.
    """
    child = {}
    for key in person1.keys():
        rand = np.random.rand()
        if rand < p:
            child[key] = person1[key]
        else:
            child[key] = person2[key]
            
    return child

def mutate_child(child, mutation=0.5):
    child = child.copy()
    for key in child.keys():
        rand = np.random.rand()
        if rand < mutation:
            child[key] = np.random.choice(parameter_grid[key])
            
    return child

In [ ]:
parameter_grid = {'min_samples_leaf': np.arange(1, 100 + 1),
                  'max_features': np.arange(1, len(perth_train.columns) - 1, 5)}

In [ ]:
initial_population = [{key: np.random.choice(val) for key, val in parameter_grid.items()}
                      for _ in range(20)]

Here I intentionally started of with an inital population where the `min_sample_leaf` is all very high. But, based on what we know of random forests, we should know that we want `min_sample_leaf` to be small. So if there was no mutuation, and only exploitation, we should expect that the genetic drift should drift to large values of `min_sample_leaf`. So lets see if mutation can actually prevent this genetic drift.

In [ ]:
generation1 = generate_next_population(initial_population, train_data, valid_data, n_children=20, mutation=0.5)

Scoring Population
1 of 20: DNA {'min_samples_leaf': 21, 'max_features': 151} has score 0.11538892000627582
2 of 20: DNA {'min_samples_leaf': 50, 'max_features': 246} has score 0.12891387539840599
3 of 20: DNA {'min_samples_leaf': 60, 'max_features': 276} has score 0.13272264547129287
4 of 20: DNA {'min_samples_leaf': 13, 'max_features': 271} has score 0.10981016146979641
5 of 20: DNA {'min_samples_leaf': 15, 'max_features': 116} has score 0.11360288593478063
6 of 20: DNA {'min_samples_leaf': 90, 'max_features': 121} has score 0.13804981290326476
7 of 20: DNA {'min_samples_leaf': 59, 'max_features': 236} has score 0.13185571519911496
8 of 20: DNA {'min_samples_leaf': 70, 'max_features': 131} has score 0.1349720843575971
9 of 20: DNA {'min_samples_leaf': 24, 'max_features': 196} has score 0.11696525912713024
10 of 20: DNA {'min_samples_leaf': 11, 'max_features': 116} has score 0.10999899715307844
11 of 20: DNA {'min_samples_leaf': 90, 'max_features': 131} has score 0.1383872030694755
12

In [ ]:
generation2 = generate_next_population(generation1, train_data, valid_data, n_children=20, mutation=0.5)

Scoring Population
1 of 20: DNA {'min_samples_leaf': 37, 'max_features': 211} has score 0.12397019457926041
2 of 20: DNA {'min_samples_leaf': 49, 'max_features': 266} has score 0.13024105426353075
3 of 20: DNA {'min_samples_leaf': 4, 'max_features': 266} has score 0.1034871133408857
4 of 20: DNA {'min_samples_leaf': 21, 'max_features': 151} has score 0.11538892000627582
5 of 20: DNA {'min_samples_leaf': 50, 'max_features': 246} has score 0.12891387539840599
6 of 20: DNA {'min_samples_leaf': 95, 'max_features': 246} has score 0.1397793346952394
7 of 20: DNA {'min_samples_leaf': 16, 'max_features': 196} has score 0.11210459663541276
8 of 20: DNA {'min_samples_leaf': 72, 'max_features': 116} has score 0.13543117903324753
9 of 20: DNA {'min_samples_leaf': 69, 'max_features': 31} has score 0.18036857577079077
10 of 20: DNA {'min_samples_leaf': 37, 'max_features': 266} has score 0.1241478037430166
11 of 20: DNA {'min_samples_leaf': 49, 'max_features': 26} has score 0.18187221737660067
12 of 

In [ ]:
generation3 = generate_next_population(generation2, train_data, valid_data, n_children=20, mutation=0.5)

Scoring Population
1 of 20: DNA {'min_samples_leaf': 58, 'max_features': 151} has score 0.13059337464275864
2 of 20: DNA {'min_samples_leaf': 26, 'max_features': 101} has score 0.12117516382749478
3 of 20: DNA {'min_samples_leaf': 4, 'max_features': 271} has score 0.10395497080216104
4 of 20: DNA {'min_samples_leaf': 35, 'max_features': 176} has score 0.1219392934481044
5 of 20: DNA {'min_samples_leaf': 86, 'max_features': 266} has score 0.13860797602538297
6 of 20: DNA {'min_samples_leaf': 21, 'max_features': 151} has score 0.11538892000627582
7 of 20: DNA {'min_samples_leaf': 77, 'max_features': 51} has score 0.15297115205966091
8 of 20: DNA {'min_samples_leaf': 26, 'max_features': 181} has score 0.11845698748859514
9 of 20: DNA {'min_samples_leaf': 72, 'max_features': 271} has score 0.13560644104105357
10 of 20: DNA {'min_samples_leaf': 89, 'max_features': 71} has score 0.14579861317730688
11 of 20: DNA {'min_samples_leaf': 37, 'max_features': 211} has score 0.12397019457926041
12 o

Very cool, in 3 generations we now have 3 people with low `min_samples_leaf`. So now we are at a point where we want the algorithm to exploit this new information it found. But because the mutation rate is so high, it is likely that `min_sample_leaf` will mutate to something bigger.

In [ ]:
generation4_high_mutation = generate_next_population(generation3, train_data, valid_data, 
                                                     n_children=20, mutation=0.5)

Scoring Population
1 of 20: DNA {'min_samples_leaf': 57, 'max_features': 101} has score 0.13196686441543531
2 of 20: DNA {'min_samples_leaf': 36, 'max_features': 271} has score 0.12343741080081441
3 of 20: DNA {'min_samples_leaf': 47, 'max_features': 116} has score 0.12787457591718618
4 of 20: DNA {'min_samples_leaf': 1, 'max_features': 271} has score 0.10260411007269372
5 of 20: DNA {'min_samples_leaf': 4, 'max_features': 176} has score 0.10313711470499154
6 of 20: DNA {'min_samples_leaf': 36, 'max_features': 181} has score 0.12383760174700248
7 of 20: DNA {'min_samples_leaf': 4, 'max_features': 271} has score 0.10395497080216104
8 of 20: DNA {'min_samples_leaf': 26, 'max_features': 266} has score 0.11818852404011145
9 of 20: DNA {'min_samples_leaf': 4, 'max_features': 146} has score 0.10336221724165372
10 of 20: DNA {'min_samples_leaf': 19, 'max_features': 266} has score 0.11344773150806396
11 of 20: DNA {'min_samples_leaf': 77, 'max_features': 271} has score 0.13644383622135833
12 o

In [ ]:
for x in generation4_high_mutation:
    print(x)

{'min_samples_leaf': 26, 'max_features': 266}
{'min_samples_leaf': 42, 'max_features': 136}
{'min_samples_leaf': 92, 'max_features': 161}
{'min_samples_leaf': 26, 'max_features': 161}
{'min_samples_leaf': 26, 'max_features': 36}
{'min_samples_leaf': 97, 'max_features': 266}
{'min_samples_leaf': 89, 'max_features': 271}
{'min_samples_leaf': 4, 'max_features': 146}
{'min_samples_leaf': 26, 'max_features': 211}
{'min_samples_leaf': 93, 'max_features': 266}
{'min_samples_leaf': 19, 'max_features': 266}
{'min_samples_leaf': 1, 'max_features': 26}
{'min_samples_leaf': 6, 'max_features': 266}
{'min_samples_leaf': 26, 'max_features': 266}
{'min_samples_leaf': 4, 'max_features': 111}
{'min_samples_leaf': 41, 'max_features': 271}
{'min_samples_leaf': 65, 'max_features': 266}
{'min_samples_leaf': 4, 'max_features': 146}
{'min_samples_leaf': 89, 'max_features': 271}
{'min_samples_leaf': 1, 'max_features': 271}


In [ ]:
generation4_low_mutation = generate_next_population(generation3, train_data, valid_data, 
                                                    n_children=20, mutation=0.1)

Scoring Population
1 of 20: DNA {'min_samples_leaf': 57, 'max_features': 101} has score 0.13196686441543531
2 of 20: DNA {'min_samples_leaf': 36, 'max_features': 271} has score 0.12343741080081441
3 of 20: DNA {'min_samples_leaf': 47, 'max_features': 116} has score 0.12787457591718618
4 of 20: DNA {'min_samples_leaf': 1, 'max_features': 271} has score 0.10260411007269372
5 of 20: DNA {'min_samples_leaf': 4, 'max_features': 176} has score 0.10313711470499154
6 of 20: DNA {'min_samples_leaf': 36, 'max_features': 181} has score 0.12383760174700248
7 of 20: DNA {'min_samples_leaf': 4, 'max_features': 271} has score 0.10395497080216104
8 of 20: DNA {'min_samples_leaf': 26, 'max_features': 266} has score 0.11818852404011145
9 of 20: DNA {'min_samples_leaf': 4, 'max_features': 146} has score 0.10336221724165372
10 of 20: DNA {'min_samples_leaf': 19, 'max_features': 266} has score 0.11344773150806396
11 of 20: DNA {'min_samples_leaf': 77, 'max_features': 271} has score 0.13644383622135833
12 o

In [ ]:
for x in generation4_low_mutation:
    print(x)

{'min_samples_leaf': 4, 'max_features': 271}
{'min_samples_leaf': 44, 'max_features': 271}
{'min_samples_leaf': 4, 'max_features': 146}
{'min_samples_leaf': 4, 'max_features': 271}
{'min_samples_leaf': 15, 'max_features': 271}
{'min_samples_leaf': 1, 'max_features': 271}
{'min_samples_leaf': 1, 'max_features': 266}
{'min_samples_leaf': 16, 'max_features': 271}
{'min_samples_leaf': 16, 'max_features': 196}
{'min_samples_leaf': 4, 'max_features': 266}
{'min_samples_leaf': 1, 'max_features': 271}
{'min_samples_leaf': 1, 'max_features': 271}
{'min_samples_leaf': 16, 'max_features': 196}
{'min_samples_leaf': 19, 'max_features': 161}
{'min_samples_leaf': 16, 'max_features': 146}
{'min_samples_leaf': 15, 'max_features': 176}
{'min_samples_leaf': 16, 'max_features': 266}
{'min_samples_leaf': 26, 'max_features': 266}
{'min_samples_leaf': 26, 'max_features': 161}
{'min_samples_leaf': 1, 'max_features': 271}


We can see that with the low mutation rate, the next generation slowly drifts to using small values of `min_samples_leaf`, since in `generation4_low_mutation` we have 9 people that have `min_samples_leaf < 5`, while `generation4_high_mutation` only has 5.

In [ ]:
generation5 = generate_next_population(generation4_low_mutation, train_data, valid_data, n_children=20, mutation=0.1)

Scoring Population
1 of 20: DNA {'min_samples_leaf': 4, 'max_features': 271} has score 0.10395497080216104
2 of 20: DNA {'min_samples_leaf': 44, 'max_features': 271} has score 0.12714005659568547
3 of 20: DNA {'min_samples_leaf': 4, 'max_features': 146} has score 0.10336221724165372
4 of 20: DNA {'min_samples_leaf': 4, 'max_features': 271} has score 0.10395497080216104
5 of 20: DNA {'min_samples_leaf': 15, 'max_features': 271} has score 0.1111275363614617
6 of 20: DNA {'min_samples_leaf': 1, 'max_features': 271} has score 0.10260411007269372
7 of 20: DNA {'min_samples_leaf': 1, 'max_features': 266} has score 0.10309294850322123
8 of 20: DNA {'min_samples_leaf': 16, 'max_features': 271} has score 0.11190263569393659
9 of 20: DNA {'min_samples_leaf': 16, 'max_features': 196} has score 0.11210459663541276
10 of 20: DNA {'min_samples_leaf': 4, 'max_features': 266} has score 0.1034871133408857
11 of 20: DNA {'min_samples_leaf': 1, 'max_features': 271} has score 0.10260411007269372
12 of 20:

Note here that we only used 5 generations here, with 20 models each time. So a total of 100 models sampled. So lets compare this to what we did in preivous weeks where we sampled 159 models

In [ ]:
%%time
best_parameters = {'min_samples_leaf': 1, 'max_features': 271}

final_model = RandomForestRegressor(criterion='squared_error', random_state=0, **best_parameters)
final_model.fit(perth_train.drop('log10_price', axis=1), perth_train['log10_price'])

y = perth_test['log10_price']
y_pred = final_model.predict(perth_test.drop('log10_price', axis=1))

print(np.mean((y - y_pred) ** 2))

0.007931673841989126
CPU times: user 56.6 s, sys: 76.6 ms, total: 56.6 s
Wall time: 56.7 s


In [ ]:
# Coordinate Descent
base_parameters = {'min_samples_leaf': 1, 'max_features': 106}

final_model = RandomForestRegressor(criterion='squared_error', random_state=0, **base_parameters)
final_model.fit(perth_train.drop('log10_price', axis=1), perth_train['log10_price'])

y = perth_test['log10_price']
y_pred = final_model.predict(perth_test.drop('log10_price', axis=1))
print(np.mean((y - y_pred) ** 2))


0.00781962457241284


In [ ]:
# Random Search
best_parameters = {'min_samples_leaf': 1, 'max_features': 191}

final_model = RandomForestRegressor(criterion='squared_error', random_state=0, **best_parameters)
final_model.fit(perth_train.drop('log10_price', axis=1), perth_train['log10_price'])

y = perth_test['log10_price']
y_pred = final_model.predict(perth_test.drop('log10_price', axis=1))

print(np.mean((y - y_pred) ** 2))

0.007857488928888677


So clearly it's not even better, and we put in all this work only to give worse results than random search! So it might not always be good, so just stick to random search. But if you do want to use genetic algorithms, do know that it can work well. But it requires a lot of effort and a lot of playing around with the parameters for it to work well.

As a general rule of thumb
- You want to start of will a very big population size - Here we start of with 20, ideally it should be more like 50-100. But as you go through generations you can slowly decrease this population size.
- You want a small chance of mutation - 50% is way too big and < 10% is better. I set 50% here because the small population size requires a high mutation rate to escape the population drift. But if you set the initial population size to be high, you most likely won't need a high mutation rate. It will be easier to escape the population drift.
- Better mutation - Here I set mutation to just randomly select a value. So if the current value of `min_samples_leaf=1` then the mutation can just make it to `min_samples_leaf=100`. But that mutation might be too crazy, and instead you want to try something more closer to 1 (so a more local exploration, rather than a global exploration). 
- Better breeding - Here we simply use the cross-over method, which is simple. But might not be as good as other breeding methods you can think of.

# Implementation Note

Basically all of the code here should really be a class. I only write it this way because it is a bit easier to understand for new programmers. But if you really care about how to write this

1. Each "DNA" should be a class, where you have one attribute as the score of the DNA, and another attribute is the gene sequence
2. The core code of GA (generating the next population, mutation, generating children, sorting population) should all be a class as well
